In [1]:
fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/output/T5_base_prefix_text2cypher_with_cell_value/predictions_eval_72.72544080604534.json'

In [2]:
schema_fpath = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json'
import json
with open(schema_fpath, 'r', encoding="utf-8") as f:
    schema = json.load(f)
    print(len(schema))

155


In [44]:
import json

with open(fpath, 'r', encoding="utf-8") as f:
    data = json.load(f)
    print(len(data))
    for each in data:
        # print(each)
        preds = each['prediction']
        preds = preds.replace(" ` ", "`")
        print(f'preds: {preds}')
        mentioned_tags = []
        normalised_db_flag = normalised_tag_flag = normalised_property_flag = False
        tag_span_start_id=tag_span_end_id = 0
        prop_start_id=prop_end_id = 0
        targeted_properties = []

        for id, c in enumerate(preds):
            if c == '`': 
                # the indicator starting postprocessing the predicted query, 
                # including the space sitting ahead of the graph nodes/edges and the case of property names regarding targeted labels/types
                if tag_span_start_id!=0:
                    tag_span_end_id=id
                    print("check it out", preds[tag_span_start_id+1: tag_span_end_id])
                    db, tag = preds[tag_span_start_id+1: tag_span_end_id].split('.')
                    print(db, tag)
                    mentioned_tags.append(tag)
                    alias = tag.lower() #TODO

                    if db in schema:
                        # print(schema[db])
                        lower_cased_tags = [original_tag.lower() for original_tag in schema[db]['tag_names_original']]
                        # 1) check the case sensitivity of tag names
                        if tag.lower() in lower_cased_tags:
                            tag_idx = lower_cased_tags.index(tag.lower())
                            # normalised the existing predicted tag names.
                            preds = preds.replace(preds[tag_span_start_id+1:tag_span_end_id], '{}.{}'.format(db, schema[db]['tag_names_original'][tag_idx]))
                            property_names = schema[db]['property_names']
                            targeted_properties = [ prop[1]  for prop in property_names if prop[0]==tag_idx]
                            print(f'targeted_properties: {targeted_properties}')
                            # 2) check if any targeted property name is predicted and its capitalization. 
                            print(preds)
                    starttag_span_start_id_id=tag_span_end_id=0
                else:
                    tag_span_start_id = id
            if len(mentioned_tags)!=0:
                if id+len(mentioned_tags[-1])+1<=len(preds):
                    if c=='.' and preds[id+len(mentioned_tags[-1])+1]!='`':
                        # the indicator of the span of property name
                        prop_start_id=id+1
                    if c==','and preds[id+len(mentioned_tags[-1])+1]!='`':
                        if preds[id-1]==')':
                            prop_end_id=id-1
                        else:
                            prop_end_id=id
                        mentioned_property = preds[prop_start_id:prop_end_id]
                        print(f'mentioned_property: {mentioned_property}')
                        lower_case_property_names = [prop.lower() for prop in targeted_properties]
                        if mentioned_property.lower() in lower_case_property_names:
                            normalised_mentioned_property = targeted_properties[lower_case_property_names.index(mentioned_property.lower())]
                        preds = preds.replace(preds[prop_start_id:prop_end_id], normalised_mentioned_property)
                        print(f'normalised preds: {preds}')
                        prop_start_id=prop_end_id=0
        # print(mentioned_tags)

491
preds: match (singer:`concert_singer.singer`) return count(*)
check it out concert_singer.singer
concert_singer singer
targeted_properties: ['Country', 'Age', 'Name', 'Song_Name', 'Is_male', 'Singer_ID', 'Song_release_year']
match (singer:`concert_singer.singer`) return count(*)
preds: match (singer:`concert_singer.singer`) return count(*)
check it out concert_singer.singer
concert_singer singer
targeted_properties: ['Country', 'Age', 'Name', 'Song_Name', 'Is_male', 'Singer_ID', 'Song_release_year']
match (singer:`concert_singer.singer`) return count(*)
preds: match (singer:`concert_singer.singer`) return singer.name,singer.country,singer.age order by singer.age desc
check it out concert_singer.singer
concert_singer singer
targeted_properties: ['Country', 'Age', 'Name', 'Song_Name', 'Is_male', 'Singer_ID', 'Song_release_year']
match (singer:`concert_singer.singer`) return singer.name,singer.country,singer.age order by singer.age desc
mentioned_property: name
normalised preds: match

ValueError: too many values to unpack (expected 2)